In [1]:
import requests
import re
import pymysql
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

In [2]:
def savenews(title,url,department,publishdate,count,content,piclinks):
    #pysql.connect(数据库URL，用户名，密码，数据库名)
    db = pymysql.connect("localhost","root","密码","数据库名",charset="utf8")
    cursor = db.cursor()
    try:
        cursor.execute('INSERT INTO cucnews(title,url,department,publishdate,count,content,piclinks) VALUES(%s,%s,%s,%s,%s,%s,%s)', (title,url,department,publishdate,count,content,piclinks))
        db.commit()
    except:
        print('数据库连接错误！')
        db.rollback()
    db.close()

In [3]:
def parsecontent(newsurl):
    try:
        option = webdriver.ChromeOptions()
        option.add_argument('headless')
        bro = webdriver.Chrome(options = option,executable_path=r"C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")
        bro.get(newsurl)
        html_source = bro.page_source
        soup = bs(html_source,"html.parser")
        
        #新闻标题
        h1 = soup.find_all("h1")
        
        info = soup.find_all("span",attrs={"class":"arti-name"})
        infotxt = info[0].get_text()       
        department = infotxt[infotxt.find('来源：')+3:infotxt.find('20')] #发表部门
        newsdate = infotxt[infotxt.find('20'):infotxt.find('20')+10] #发表日期
        r = re.compile('[\u4e00-\u9fa5]+')
        dep = r.findall(department)
        
        countinfo = soup.find_all("span",attrs={"class":"WP_VisitCount"})
        count = countinfo[0].get_text()
        
        all_image = soup.find("article",attrs={"class":"con-area"}).find_all('img')
        #print(all_image[0][all_image[0].find('original-src="'):all_image[0].find('"')])
        piclinks = ''
        for i in range(len(all_image)):
            if all_image[i] != '':
                link = 'http://www.cuc.edu.cn/_upload/'+str(all_image[i])[str(all_image[i]).find('src="')+14:str(all_image[i]).find('.')+4]
                piclinks = piclinks + link + ' '
        
        contentinfo = soup.find_all("div",attrs={"class":"wp_articlecontent"})
        content = contentinfo[0].get_text()
        
        bro.quit()
    except:
        print('data-error')
    
    return dep[0],newsdate,count,piclinks,content

In [4]:
parsecontent("http://www.cuc.edu.cn/news/2021/0311/c1901a178638/page.htm")

('科学研究处',
 '2021-03-11',
 '14\n',
 'http://www.cuc.edu.cn/_upload/article/images/ee/bb/96da96e3431e8c04020d62aa52e1/2c06c7dc-c227-4027-a88f-8f387d8b815f.png ',
 '（通讯员 王珏 闫蕾）近日，在科学研究处的积极推进下，我校持续引入中国共产党思想理论资源数据库。该数据库经一年试用受到了师生广泛欢迎，其持续订购将助益学校师生党史学习和相关科研需求。中国共产党思想理论资源数据库是在中央领导同志的亲切关怀和中宣部、新闻出版广电总局的有力指导下，由人民出版社开发建成，被党政干部和专家学者称为用科学技术传播中国化马克思主义的重大创新工程，目前收入图书分为14个子库，13000多册、7000多万个知识点。师生登录学校图书馆网站，在数字资源的中文数据库中即可找到并使用该库（http://libw.cuc.edu.cn/2018/1110/c1195a19710/page.htm）。中国共产党思想理论资源数据库完整系统收入了党的思想理论主要著作文献，内容覆盖我国出版的所有马列经典著作、党和国家主要领导人著作、公开发表的中央文件文献、国家法律法规和党的思想理论领域知识点，并代表性收入了大量研究性著作、党史和国际共运史著作、重要人物资料以及革命战争年代出版的部分重要图书等。中国共产党思想理论资源数据库的引入为我校师生开展各类科学研究工作提供了有力支持，为开展对习近平新时代中国特色社会主义思想进行原创性、学理化、学科化研究阐释以及对新时代治国理政实践中提出的具有原创性、时代性、指导性重大思想观点的研究阐释提供了丰富信息，为推动高校加快中国特色哲学社会科学学科体系、学术体系、话语体系建设提供了数据资源。科学研究处始终坚持贯彻党的教育方针和学校的发展规划，在科学研究服务工作中坚持正确的政治方向、价值取向、研究导向，以“三大体系”构建为引领，以系统治理、创新驱动为根本动力，积极服务学校科研工作的高质量发展。（编辑：阎玺）')

In [ ]:
urlpath="http://www.cuc.edu.cn/news/1901/list"
try:
    for i in range(1,4):
        url=urlpath+str(i)+".htm"
        #print(url)
        r=requests.get(url)
        code=r.encoding
        content=r.content
        rt=str(content,"utf-8")
        soup=bs(rt,"html.parser")
        href=soup.find_all("h3",attrs={'class','tit'})
        for h in href:
            newsurl=h.find_all('a')
            urllen=str(newsurl[0]).find('target')
            newsurl=str(newsurl[0])[9:urllen-2]
            newsurl="http://www.cuc.edu.cn"+newsurl
            title=h.get_text()
            print(title) 
            print("------Save Start!---------")
            department,date,count,piclinks,content = parsecontent(newsurl)
            savenews(title,newsurl,department,date,count,content,piclinks)
            print("------Save Sucess！---------")
            #savenews(title,newsurl)
except:
    print("Error!")


我校持续引入中国共产党思想理论资源数据库

------Save Start!---------
------Save Sucess！---------

宏图在胸 扬帆奋进 | 中国传媒大学2021年春季工作会议召开

------Save Start!---------
------Save Sucess！---------

我校获批国家新闻出版署重点实验室

------Save Start!---------
------Save Sucess！---------

我校广告学院推出纪录片《中国广告四十年》

------Save Start!---------
------Save Sucess！---------

中国传媒大学新增16个“双万计划”一流本科专业建设点

------Save Start!---------
------Save Sucess！---------

我校女教授协会第四届理事会第一次会议暨青年女教师发展协会成立...

------Save Start!---------
------Save Sucess！---------

我校学生参与2022冬奥会“相约北京”冬季体育系列测试活动

------Save Start!---------
------Save Sucess！---------

我校与英国阿伯泰大学举行线上合作会议

------Save Start!---------
------Save Sucess！---------

我校师生高度关注全国“两会”开幕

------Save Start!---------
------Save Sucess！---------

我校获3项教育部第八届高等学校科学研究优秀成果奖

------Save Start!---------
------Save Sucess！---------

新华社北京分社副社长李斌来融媒体中心调研交流

------Save Start!---------
------Save Sucess！---------

刘守训副校长到离退休工作处调研

------Save Start!---------
------Save Sucess！---------

研究生网络授课平稳运行 融合理念深入人心

------Save 